# YouTube on Android

The goal of this experiment is to run Youtube videos on a Pixel device running Android and collect results.

In [1]:
from conf import LisaLogging
import logging
LisaLogging.setup(level=logging.DEBUG)

2018-08-17 16:28:31,996 INFO    : root         : Using LISA logging configuration:
2018-08-17 16:28:31,999 INFO    : root         :   /home/steven/lisa/lisa-github/logging.conf


In [2]:
%pylab inline

import json
import os

# Support to access the remote target
import devlib
from env import TestEnv

# Import support for Android devices
from android import System, Screen, Workload

# Support for trace events analysis
from trace import Trace

# Suport for FTrace events parsing and visualization
import trappy

import pandas as pd
import sqlite3

Populating the interactive namespace from numpy and matplotlib


2018-08-17 16:28:33,054 DEBUG   : JsonConf     : loading JSON...
2018-08-17 16:28:33,056 DEBUG   : JsonConf     : Loaded JSON configuration:
2018-08-17 16:28:33,058 DEBUG   : JsonConf     :    {u'nrg_model': {u'big': {u'cluster': {u'nrg_max': 112}, u'cpu': {u'cap_max': 1024, u'nrg_max': 670}}, u'little': {u'cluster': {u'nrg_max': 112}, u'cpu': {u'cap_max': 1024, u'nrg_max': 670}}}, u'board': {u'big_core': u'A53_1', u'cores': [u'A53_0', u'A53_0', u'A53_0', u'A53_0', u'A53_1', u'A53_1', u'A53_1', u'A53_1'], u'modules': [u'bl', u'cpufreq', u'cpuidle']}}
2018-08-17 16:28:33,062 DEBUG   : JsonConf     : loading JSON...
2018-08-17 16:28:33,064 DEBUG   : JsonConf     : Loaded JSON configuration:
2018-08-17 16:28:33,065 DEBUG   : JsonConf     :    {u'board': {u'big_core': u'A57', u'cores': [u'A53', u'A53', u'A53', u'A53', u'A57', u'A57'], u'modules': [u'bl', u'cpufreq']}}
2018-08-17 16:28:33,067 DEBUG   : JsonConf     : loading JSON...
2018-08-17 16:28:33,069 DEBUG   : JsonConf     : Loaded JS

## Support Functions

This function helps us run our experiments:

In [3]:
def experiment():
    
    # Configure governor
    target.cpufreq.set_all_governors('schedutil')

    # Get workload
    wload = Workload(te).getInstance(te, 'YouTube')
    
    # Run Youtube workload
    wload.run(te.res_dir, 'https://youtu.be/XSGBVzeBUbk?t=45s',
              video_duration_s=5, collect='ftrace')

    # Dump platform descriptor
    te.platform_dump(te.res_dir)

## Test environment setup
For more details on this please check out **examples/utils/testenv_example.ipynb**.

**devlib** requires the ANDROID_HOME environment variable configured to point to your local installation of the Android SDK. If you have not this variable configured in the shell used to start the notebook server, you need to run a cell to define where your Android SDK is installed or specify the ANDROID_HOME in your target configuration.

In case more than one Android device are conencted to the host, you must specify the ID of the device you want to target in **my_target_conf**. Run **adb devices** on your host to get the ID.

In [4]:
# Setup target configuration
androidsdk_path=os.path.join(os.getcwd(), "../../../../android-sdk")
my_conf = {
    "platform"    : 'android',
    "board"       : "hikey960",
    "device" : "0123456789ABCDEF",
    
    "ANDROID_HOME" : androidsdk_path,
    "rtapp-calib" : {"0": 302, "1": 302, "2": 304, "3": 304, "4": 136, "5": 137, "6": 136, "7": 136},

     "emeter" : {
        "instrument" : "acme",
        "conf" : {
            # Absolute path to the iio-capture binary on the host
            'iio-capture' : '/usr/bin/iio-capture',
            # Default host name of the BeagleBone Black
            'ip_address'     : '10.65.34.1',
        },
        "channel_map" : {
            "Device0" : 0, # iio:device0
            "Device1" : 1, # iio:device0
        }
    },


    # Folder where all the results will be collected
    "results_dir" : "Youtube_example",

    # Define devlib modules to load
    "modules"     : [
        'cpufreq',       # enable CPUFreq support
    ],

    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_wakeup",
            "sched_wakeup_new",
            "sched_overutilized",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "cpu_capacity",
            "cpu_frequency",
             
            # Add here the events you wanna enable by hand
            "sched_energy_diff",
         ],
         "buffsize" : 100 * 1024,
    },

    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'taskset' ],
}

In [5]:
# Initialize a test environment using:
te = TestEnv(my_conf)
target = te.target

2018-08-17 16:28:33,594 INFO    : TestEnv      : Using base path: /home/steven/lisa/lisa-github
2018-08-17 16:28:33,596 INFO    : TestEnv      : Loading custom (inline) target configuration
2018-08-17 16:28:33,598 WARNING : TestEnv      : Wipe previous contents of the results folder:
2018-08-17 16:28:33,600 WARNING : TestEnv      :    /home/steven/lisa/lisa-github/results/Youtube_example
2018-08-17 16:28:33,603 INFO    : TestEnv      : External tools using:
2018-08-17 16:28:33,604 INFO    : TestEnv      :    ANDROID_HOME: /home/steven/lisa/lisa-github/ipynb/workshop/android/../../../../android-sdk
2018-08-17 16:28:33,606 INFO    : TestEnv      :    CATAPULT_HOME: /home/steven/lisa/lisa-github/tools/catapult
2018-08-17 16:28:33,608 INFO    : TestEnv      : Devlib modules to load: ['bl', 'cpuidle', 'cpufreq']
2018-08-17 16:28:33,610 INFO    : TestEnv      : Connecting Android target [0123456789ABCDEF]
2018-08-17 16:28:33,612 INFO    : TestEnv      : Connection settings:
2018-08-17 16:28:

In [6]:
# ADB ROOT is requried for systrace to be able to get the generated trace from the target
!adb root

## Workloads execution

This is done using the **experiment** helper function defined above which is configured to run a **Youtube** experiment.

In [9]:
# Intialize Workloads for this test environment
results = experiment()

2018-08-17 16:32:16,018 DEBUG   : YouTube      : Workload created
2018-08-17 16:32:16,021 INFO    : Screen       : Setting screen ON
2018-08-17 16:32:18,162 INFO    : System       : am force-stop com.paraphron.youtube
2018-08-17 16:32:19,016 INFO    : Screen       : Force manual orientation
2018-08-17 16:32:19,019 INFO    : Screen       : Set orientation: PORTRAIT
2018-08-17 16:32:20,558 INFO    : Screen       : Set brightness: 0%
2018-08-17 16:32:22,607 INFO    : System       : am start -a android.intent.action.VIEW https://youtu.be/XSGBVzeBUbk?t=45s
2018-08-17 16:32:23,958 INFO    : System       : input tap 875 875
2018-08-17 16:32:24,267 INFO    : YouTube      : FTrace START
2018-08-17 16:32:28,785 DEBUG   : FtraceCollector : Trace CPUFreq frequencies
2018-08-17 16:32:28,848 DEBUG   : FtraceCollector : Trace CPUIdle states
2018-08-17 16:32:28,962 INFO    : YouTube      : Play video for 5 [s]
2018-08-17 16:32:33,971 DEBUG   : FtraceCollector : Trace CPUFreq frequencies
2018-08-17 16:

## Benchmarks results

In [ ]:
# Benchmark statistics
db_file = os.path.join(te.res_dir, "framestats.txt")
!sed '/Stats since/,/99th/!d;/99th/q' {db_file}

# For all results:
# !cat {results['db_file']}

## Traces visualisation

For more information on this please check **examples/trace_analysis/TraceAnalysis_TasksLatencies.ipynb**.

In [ ]:
# Parse all traces
platform_file = os.path.join(te.res_dir, 'platform.json')
with open(platform_file, 'r') as fh:
    platform = json.load(fh)
trace_file = os.path.join(te.res_dir, 'trace.dat')
trace = Trace(platform, trace_file, events=my_conf['ftrace']['events'])

trappy.plotter.plot_trace(trace.ftrace)

In [ ]:
try:
    trace.analysis.frequency.plotClusterFrequencies();
    logging.info('Plotting cluster frequencies for [sched]...')
except: pass

In [ ]:
df = trace.data_frame.trace_event('sched_energy_diff')

In [ ]:
df.head()